In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
transformer_name = "beit"

In [ ]:
!pip install -q datasets transformers
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import urllib.request, zipfile
# filehandle, _ = urllib.request.urlretrieve ("http://haviro.sbs/Cp/Uploads/Data.zip")
# zip_file_object = zipfile.ZipFile (filehandle, 'r')
# zip_file_object.extractall ("/content/drive/MyDrive/Colab Notebooks/")


In [ ]:
from datasets import load_dataset

# loading the dataset
ds_path = "/content/drive/MyDrive/Colab Notebooks/Data"
ds = load_dataset(path = ds_path)
ds

Resolving data files:   0%|          | 0/400 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/400 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 400
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 400
    })
})

In [ ]:
from transformers import AutoFeatureExtractor

#loading the feature extractor
if transformer_name == 'vit':
  # model_name = 'alirzb/vit-base-patch16-224'
  model_name = 'google/vit-base-patch16-224'
elif transformer_name == 'deit':
  # model_name = 'alirzb/deit-base-distilled-patch16-224'
  model_name = 'facebook/deit-base-distilled-patch16-224'
elif transformer_name == 'swint':
  # model_name = 'alirzb/swin-base-patch4-window7-224'
  model_name = 'microsoft/swin-base-patch4-window7-224'
elif transformer_name == 'beit':
  # model_name = 'alirzb/beit-base-patch16-224'
  model_name = 'microsoft/beit-base-patch16-224'
else:
  print('wrong model name')
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x.convert('RGB') for x in example_batch['image']], return_tensors='pt')
    inputs['label'] = example_batch['label']
    return inputs

# applying transform
prepared_ds = ds.with_transform(transform)

import torch

def collate_fn(batch):
  #data collator
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
  # function which calculates accuracy for a certain set of predictions
  return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

/usr/local/lib/python3.10/dist-packages/transformers/models/beit/feature_extraction_beit.py:28: FutureWarning: The class BeitFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use BeitImageProcessor instead.
  warnings.warn(


In [ ]:
labels = ds['train'].features['label'].names


if transformer_name == 'vit':
  print(transformer_name)
  from transformers import  ViTForImageClassification, Trainer
  model = ViTForImageClassification.from_pretrained(
      model_name,
      num_labels=len(labels),
      id2label={str(i): c for i, c in enumerate(labels)},
      label2id={c: str(i) for i, c in enumerate(labels)},
      ignore_mismatched_sizes = True,
  )
elif transformer_name == 'deit':
  print(transformer_name)
  from transformers import  DeiTForImageClassification, Trainer
  model = DeiTForImageClassification.from_pretrained(
      model_name,
      num_labels=len(labels),
      id2label={str(i): c for i, c in enumerate(labels)},
      label2id={c: str(i) for i, c in enumerate(labels)},
      ignore_mismatched_sizes = True,
  )
elif transformer_name == 'swint':
  print(transformer_name)
  from transformers import  SwinForImageClassification, Trainer
  model = SwinForImageClassification.from_pretrained(
      model_name,
      num_labels=len(labels),
      id2label={str(i): c for i, c in enumerate(labels)},
      label2id={c: str(i) for i, c in enumerate(labels)},
      ignore_mismatched_sizes = True,
  )
elif transformer_name == 'beit':
  print(transformer_name)
  from transformers import  BeitForImageClassification, Trainer
  model = BeitForImageClassification.from_pretrained(
      model_name,
      num_labels=len(labels),
      id2label={str(i): c for i, c in enumerate(labels)},
      label2id={c: str(i) for i, c in enumerate(labels)},
      ignore_mismatched_sizes = True,
  )
else:
  print('wrong transformer name')

beit


In [ ]:
! pip install -U accelerate
! pip install -U transformers
import accelerate
import transformers
from transformers import TrainingArguments
transformers.__version__, accelerate.__version__

('4.35.2', '0.25.0')

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 16
# Defining training arguments (set push_to_hub to false if you don't want to upload it to HuggingFace's model hub)
training_args = TrainingArguments(
    model_name,
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

# Instantiate the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["test"],
    tokenizer=feature_extractor,
)

In [ ]:
# Train and save results
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

# Evaluate on validation set
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm
test_dataloader = DataLoader(prepared_ds['test'], batch_size=32)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
model.eval()

y_true = []
y_pred = []

runtimes = []
torch.cuda.empty_cache()

for batch in tqdm(test_dataloader):

    images = batch['pixel_values'].to(device)

    import time
    start_time = time.time()

    # Forward pass
    outputs = model(images)


    end_time = time.time()
    runtimes.append(end_time - start_time)

    # Append batch predictions and labels to lists
    y_true.append(batch['label'].numpy())
    y_pred.append(np.argmax(outputs.logits.cpu().detach().numpy(), axis=1))

    del outputs
    torch.cuda.empty_cache()


y_true = np.concatenate(y_true, axis=0)
y_pred = np.concatenate(y_pred, axis=0)

100%|██████████| 13/13 [00:10<00:00,  1.28it/s]


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
import numpy as np

print(model_name)
# Compute metrics

print(f'runtime: {sum(runtimes)}')
print(f'pred per sec: {400/sum(runtimes)}')
print(f'Accuracy: {accuracy_score(y_true, y_pred):.3f}')
print(f'Confusion Matrix:\n {confusion_matrix(y_true, y_pred)}')
print("Precision, Recall, F1-score: {:.3f} - {:.3f} - {:.3f}".format(*precision_recall_fscore_support(y_true, y_pred, average="binary")))

microsoft/beit-base-patch16-224
runtime: 0.49329686164855957
pred per sec: 810.8707577486531
Accuracy: 0.990
Confusion Matrix:
 [[199   1]
 [  3 197]]
Precision, Recall, F1-score: 0.995 - 0.985 - 0.990
